# Stock Screening Strategies

This notebook demonstrates various stock screening strategies using tvscreener.

**Strategies covered:**
- Value investing (P/E, P/B, Graham criteria)
- Dividend investing
- Momentum strategies
- Technical analysis (RSI, MACD, moving averages)
- Quality screens (ROE, margins)

In [ ]:
from tvscreener import StockScreener, StockField, IndexSymbol, Market
import pandas as pd

---
## Value Investing

### Low P/E Stocks

In [ ]:
ss = StockScreener()
ss.set_index(IndexSymbol.SP500)

# Value criteria
ss.where(StockField.PE_RATIO_TTM.between(5, 15))
ss.where(StockField.PRICE_TO_BOOK_FY < 3)
ss.where(StockField.MARKET_CAPITALIZATION > 10e9)  # Large cap

ss.select(
    StockField.NAME,
    StockField.PRICE,
    StockField.PE_RATIO_TTM,
    StockField.PRICE_TO_BOOK_FY,
    StockField.DIVIDEND_YIELD_FY,
    StockField.MARKET_CAPITALIZATION
)
ss.sort_by(StockField.PE_RATIO_TTM, ascending=True)
ss.set_range(0, 500)

df = ss.get()
print(f"Found {len(df)} low P/E S&P 500 stocks")
df.head(15)

### Graham Number Criteria

Benjamin Graham's conservative value criteria.

In [ ]:
ss = StockScreener()

ss.where(StockField.PE_RATIO_TTM < 15)
ss.where(StockField.PRICE_TO_BOOK_FY < 1.5)
ss.where(StockField.CURRENT_RATIO_FY > 2)  # Strong liquidity
ss.where(StockField.TOTAL_DEBT_TO_EQUITY_FY < 0.5)  # Low debt
ss.where(StockField.EARNINGS_PER_SHARE_DILUTED_TTM > 0)  # Profitable

ss.select(
    StockField.NAME,
    StockField.PRICE,
    StockField.PE_RATIO_TTM,
    StockField.PRICE_TO_BOOK_FY,
    StockField.CURRENT_RATIO_FY,
    StockField.TOTAL_DEBT_TO_EQUITY_FY
)

df = ss.get()
print(f"Found {len(df)} Graham-style value stocks")
df.head(15)

---
## Dividend Investing

### High Yield with Sustainable Payout

In [ ]:
ss = StockScreener()
ss.set_index(IndexSymbol.SP500)

ss.where(StockField.DIVIDEND_YIELD_FY > 3)
ss.where(StockField.PAYOUT_RATIO_TTM.between(20, 70))  # Sustainable
ss.where(StockField.MARKET_CAPITALIZATION > 5e9)

ss.select(
    StockField.NAME,
    StockField.PRICE,
    StockField.DIVIDEND_YIELD_FY,
    StockField.DIVIDENDS_PER_SHARE_FY,
    StockField.PAYOUT_RATIO_TTM,
    StockField.MARKET_CAPITALIZATION
)
ss.sort_by(StockField.DIVIDEND_YIELD_FY, ascending=False)
ss.set_range(0, 500)

df = ss.get()
print(f"Found {len(df)} high dividend S&P 500 stocks")
df.head(15)

---
## Momentum Strategies

### Top Daily Gainers

In [ ]:
ss = StockScreener()

ss.where(StockField.CHANGE_PERCENT > 5)  # Up 5%+
ss.where(StockField.VOLUME >= 1_000_000)  # Liquid
ss.where(StockField.PRICE > 5)  # No penny stocks

ss.select(
    StockField.NAME,
    StockField.PRICE,
    StockField.CHANGE_PERCENT,
    StockField.VOLUME,
    StockField.RELATIVE_VOLUME
)
ss.sort_by(StockField.CHANGE_PERCENT, ascending=False)
ss.set_range(0, 50)

df = ss.get()
print(f"Top {len(df)} gainers today")
df.head(15)

### 3-Month Momentum in S&P 500

In [ ]:
ss = StockScreener()
ss.set_index(IndexSymbol.SP500)

ss.where(StockField.PERFORMANCE_3_MONTH > 15)  # Up 15%+ in 3 months
ss.where(StockField.PERFORMANCE_1_MONTH > 0)  # Still trending up

ss.select(
    StockField.NAME,
    StockField.PRICE,
    StockField.PERFORMANCE_1_MONTH,
    StockField.PERFORMANCE_3_MONTH,
    StockField.PERFORMANCE_YTD,
    StockField.RELATIVE_STRENGTH_INDEX_14
)
ss.sort_by(StockField.PERFORMANCE_3_MONTH, ascending=False)
ss.set_range(0, 500)

df = ss.get()
print(f"Found {len(df)} momentum stocks in S&P 500")
df.head(15)

---
## Technical Analysis

### RSI Oversold

In [ ]:
ss = StockScreener()
ss.set_index(IndexSymbol.SP500)

ss.where(StockField.RELATIVE_STRENGTH_INDEX_14 < 30)  # Oversold
ss.where(StockField.VOLUME >= 500_000)

ss.select(
    StockField.NAME,
    StockField.PRICE,
    StockField.CHANGE_PERCENT,
    StockField.RELATIVE_STRENGTH_INDEX_14,
    StockField.VOLUME
)
ss.sort_by(StockField.RELATIVE_STRENGTH_INDEX_14, ascending=True)
ss.set_range(0, 500)

df = ss.get()
print(f"Found {len(df)} oversold S&P 500 stocks (RSI < 30)")
df.head(15)

### Golden Cross Setup

Price above moving averages in bullish alignment.

In [ ]:
ss = StockScreener()
ss.set_index(IndexSymbol.SP500)

# Bullish alignment: Price > SMA50 > SMA200
ss.where(StockField.PRICE > StockField.SIMPLE_MOVING_AVERAGE_50)
ss.where(StockField.SIMPLE_MOVING_AVERAGE_50 > StockField.SIMPLE_MOVING_AVERAGE_200)

ss.select(
    StockField.NAME,
    StockField.PRICE,
    StockField.SIMPLE_MOVING_AVERAGE_50,
    StockField.SIMPLE_MOVING_AVERAGE_200,
    StockField.CHANGE_PERCENT
)
ss.set_range(0, 500)

df = ss.get()
print(f"Found {len(df)} stocks with golden cross setup")
df.head(15)

### MACD Bullish

In [ ]:
ss = StockScreener()
ss.set_index(IndexSymbol.SP500)

ss.where(StockField.MACD_LEVEL_12_26 > 0)  # MACD positive
ss.where(StockField.MACD_LEVEL_12_26 > StockField.MACD_SIGNAL_12_26_9)  # Above signal

ss.select(
    StockField.NAME,
    StockField.PRICE,
    StockField.MACD_LEVEL_12_26,
    StockField.MACD_SIGNAL_12_26_9,
    StockField.CHANGE_PERCENT
)
ss.set_range(0, 500)

df = ss.get()
print(f"Found {len(df)} stocks with bullish MACD")
df.head(15)

### Multi-Timeframe Analysis

Use `with_interval()` to analyze on different timeframes.

In [ ]:
ss = StockScreener()
ss.set_index(IndexSymbol.SP500)

# Daily RSI moderate
ss.where(StockField.RELATIVE_STRENGTH_INDEX_14.between(40, 60))

# Hourly RSI oversold (entry signal)
rsi_1h = StockField.RELATIVE_STRENGTH_INDEX_14.with_interval('60')
ss.where(rsi_1h < 40)

ss.select(
    StockField.NAME,
    StockField.PRICE,
    StockField.RELATIVE_STRENGTH_INDEX_14,  # Daily
    rsi_1h,  # Hourly
    StockField.CHANGE_PERCENT
)
ss.set_range(0, 500)

df = ss.get()
print(f"Found {len(df)} stocks: daily RSI neutral, hourly RSI oversold")
df.head(15)

---
## Quality Screens

### High ROE with Low Debt

In [ ]:
ss = StockScreener()
ss.set_index(IndexSymbol.SP500)

ss.where(StockField.RETURN_ON_EQUITY_TTM > 20)  # ROE > 20%
ss.where(StockField.RETURN_ON_ASSETS_TTM > 10)  # ROA > 10%
ss.where(StockField.TOTAL_DEBT_TO_EQUITY_FY < 1)  # Moderate debt

ss.select(
    StockField.NAME,
    StockField.PRICE,
    StockField.RETURN_ON_EQUITY_TTM,
    StockField.RETURN_ON_ASSETS_TTM,
    StockField.GROSS_MARGIN_TTM,
    StockField.TOTAL_DEBT_TO_EQUITY_FY
)
ss.sort_by(StockField.RETURN_ON_EQUITY_TTM, ascending=False)
ss.set_range(0, 500)

df = ss.get()
print(f"Found {len(df)} high quality S&P 500 stocks")
df.head(15)

---
## Sector Analysis

### Technology Sector

In [ ]:
ss = StockScreener()

ss.where(StockField.SECTOR == 'Electronic Technology')
ss.where(StockField.MARKET_CAPITALIZATION > 10e9)
ss.where(StockField.REVENUE_GROWTH_TTM > 10)  # Growing revenue

ss.select(
    StockField.NAME,
    StockField.PRICE,
    StockField.MARKET_CAPITALIZATION,
    StockField.REVENUE_GROWTH_TTM,
    StockField.PE_RATIO_TTM
)
ss.sort_by(StockField.MARKET_CAPITALIZATION, ascending=False)

df = ss.get()
print(f"Found {len(df)} large cap tech stocks")
df.head(15)

---
## Combined Strategy: Value + Quality + Momentum

In [ ]:
ss = StockScreener()
ss.set_index(IndexSymbol.SP500)

# Value
ss.where(StockField.PE_RATIO_TTM.between(10, 25))

# Quality
ss.where(StockField.RETURN_ON_EQUITY_TTM > 15)
ss.where(StockField.TOTAL_DEBT_TO_EQUITY_FY < 1)

# Momentum
ss.where(StockField.PERFORMANCE_3_MONTH > 0)
ss.where(StockField.RELATIVE_STRENGTH_INDEX_14.between(40, 70))

ss.select(
    StockField.NAME,
    StockField.PRICE,
    StockField.PE_RATIO_TTM,
    StockField.RETURN_ON_EQUITY_TTM,
    StockField.PERFORMANCE_3_MONTH,
    StockField.RELATIVE_STRENGTH_INDEX_14
)
ss.sort_by(StockField.RETURN_ON_EQUITY_TTM, ascending=False)
ss.set_range(0, 500)

df = ss.get()
print(f"Found {len(df)} stocks matching Value + Quality + Momentum criteria")
df.head(15)

---
## International Markets

In [ ]:
# Japanese large caps
ss = StockScreener()
ss.set_markets(Market.JAPAN)

ss.where(StockField.MARKET_CAPITALIZATION > 10e9)
ss.where(StockField.VOLUME > 100_000)

ss.select(
    StockField.NAME,
    StockField.PRICE,
    StockField.MARKET_CAPITALIZATION,
    StockField.PE_RATIO_TTM,
    StockField.CHANGE_PERCENT
)
ss.sort_by(StockField.MARKET_CAPITALIZATION, ascending=False)

df = ss.get()
print(f"Found {len(df)} Japanese large cap stocks")
df.head(15)